### Importing relevant libraries 

In [1]:
import pandas as pd
import numpy as np
import random

In [172]:
## This function select random values and tries to convert them to 100%
## This makes the values sum up to 100
## It also gives decimal values which when rounded might be +/- 1 of 100.

def random_weights(total_sum):
    nums = np.random.randint(100, size=6)
    return np.round(nums/np.sum(nums)*total_sum).astype(int)


n = random_weights(100)
print(n)
print(n.sum())

[26 20  8  8 28 10]
100


In [735]:
## This function also samples random integers from a range of values
## Then computes the sum to assert it is 100.
## Then returns the pick... This could take very long.

def get_scores(n):
    x = 200
    while True:
        pick = random.sample([i for i in range(5, 65, 5)], n)
        if sum(pick) <= x:
            #break
            return pick

In [736]:
for i in range(8):
    np.random.seed(i)
    print(np.sort(get_scores(8)))

[ 5 10 15 20 25 30 35 60]
[ 5 10 15 20 25 30 40 50]
[ 5 10 15 20 25 35 40 45]
[ 5 10 15 20 25 30 40 50]
[ 5 10 15 20 25 30 35 55]
[ 5 10 15 20 25 30 35 50]
[ 5 10 15 20 25 30 40 55]
[ 5 10 15 20 25 35 40 45]


In [507]:
def random_with_patterns(n, m):
    
    """
    n: Number of less difficult methods.
    m: Number of more difficult methods.    
    """
    
    ## Generate random values from a lower range for methods with less difficulty
    list1 = list(np.random.randint(low = 5, high = 50, size = n))
    
    ## Generate random values from a higher range for more difficult verification methods
    list2 = list(np.random.randint(low = 30, high = 60, size=m))
    
    ## Join both list
    for i in list2:
        list1.append(i)
        
    ## Convert them, such that they sum up to 100
    list_ = [round(i/sum(list1) * 100, 2) for i in list1]
    
    ## Handling the errors due to approximation during rounding up.
    if sum(list_) > 100:
        
        ## A random index where the correction will be made.
        my_index = np.random.randint(len(list_))
        
        ## Subtract the excessive value from the selected index
        list_[my_index] = list_[my_index] - round((sum(list_) - 100), 2)
        
        ## Returns the list
        return list_
    
    ## Handling values that sums up to less than 100 after rounding up
    elif sum(list_) < 100:
        
        ## Select random index.
        my_index = np.random.randint(len(list_))
        
        ## Add the remaining to the selected index.
        list_[my_index] = list_[my_index] + round((100 - sum(list_)), 2)
        
        ## Return the list
        return list_
    
    ## The value equals 100
    else:
        return list_

In [28]:
## Import dataset here

data = pd.read_csv("File path")

In [117]:
df = data.copy()

In [642]:
df1.shape

(532, 12)

In [121]:
# drop missing values
df.dropna(axis = 0, inplace=True)

In [122]:
df.shape

(757, 12)

In [120]:
# drop columns that are not really important for this analysis
df.drop(["Confidence (low, so-so, high)", "public_gists", "public_repos",
        "followers", "following", "information_count", "account_age", 
        "time_since_updated", "is_idena_connected"], axis=1, inplace=True)

In [33]:
df.columns

Index(['handle', 'Is Sybil? (T or F)', 'trust_bonus', 'is_brightid_verified',
       'is_duniter_verified', 'is_twitter_verified', 'is_poap_verified',
       'is_ens_verified', 'is_google_verified', 'is_facebook_verified',
       'is_poh_verified', 'is_idena_verified'],
      dtype='object')

In [123]:
## Resetting the index of the data
df.reset_index(drop=True, inplace=True)

In [379]:
## With duniter
trust_scores = []

trust_mean_differences = []

trust_median_differences = []

sybil_mean = []

not_sybil_mean = []

sybil_median = []

not_sybil_median = []

def calc_score(dfs):
    
    dff = dfs.copy()
    
    scores = random_with_patterns(5, 4)
    
    trust_scores.append([i for i in scores])
    
    total_score = []
    
    dff.is_brightid_verified = np.where(dff.is_brightid_verified == True, scores[0], 0)
    
    dff.is_duniter_verified = np.where(dff.is_duniter_verified == True, scores[1], 0)

    dff.is_twitter_verified = np.where(dff.is_twitter_verified == True, scores[2], 0)
    
    dff.is_google_verified = np.where(dff.is_google_verified == True, scores[3], 0)
    
    dff.is_facebook_verified = np.where(dff.is_facebook_verified == True, scores[4], 0)
    
    dff.is_poap_verified = np.where(dff.is_poap_verified == True, scores[5], 0)
    
    dff.is_ens_verified = np.where(dff.is_ens_verified == True, scores[6], 0)
    
    dff.is_poh_verified = np.where(dff.is_poh_verified == True, scores[7], 0)
    
    dff.is_idena_verified = np.where(dff.is_idena_verified == True, scores[8], 0)

    
    for row in range(dff.shape[0]):
        total_score.append(sum(dff.loc[row, ["is_brightid_verified", "is_duniter_verified", "is_twitter_verified", "is_poap_verified",
                             "is_ens_verified", "is_google_verified", 'is_facebook_verified', 'is_poh_verified', 'is_idena_verified']]))
    
    dff["total_score"] = total_score
    
    sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[1]
    not_sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[0]
    
    sybil_mean.append(sybil_mean_)
    not_sybil_mean.append(not_sybil_mean_)
    
    sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[1]
    not_sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[0]
    
    sybil_median.append(sybil_median_)
    not_sybil_median.append(not_sybil_median_)
    
    trust_median_differences.append(abs(sybil_median_ - not_sybil_median_))
    
    trust_mean_differences.append(abs(sybil_mean_ - not_sybil_mean_))

In [ ]:
# Changing all columns with the verification platforms to integer data type
for column in df.columns[3:]:
    df[column] = df[column].astype(int) 

In [380]:
## To change the random seeds to control the number of iteration and to make sure there is no duplicates.
for i in range(15):
    np.random.seed(i)
    calc_score(df)

In [381]:
## Convert the output to a dataframe for easy overview.
diff = pd.DataFrame()
diff["Combination of weights"] = trust_scores
diff["Sybil mean"] = sybil_mean
diff["not_sybil mean"] = not_sybil_mean
diff["Mean differences"] = trust_mean_differences
diff["Sybil median"] = sybil_median
diff["not_sybil median"] = not_sybil_median
diff["Median differences"] = trust_median_differences


In [382]:
diff.sort_values(by="Mean differences", ascending=False).reset_index(drop=True)

,Combination of weights,Sybil mean,not_sybil mean,Mean differences,Sybil median,not_sybil median,Median differences
0,"[5, 3, 13, 8, 15, 14, 14, 13, 15]",18.935933,30.429648,11.493715,13,35,22
1,"[14, 6, 4, 9, 15, 15, 13, 12, 12]",20.696379,31.972362,11.275983,14,37,23
2,"[7, 4, 6, 11, 11, 15, 16, 12, 16]",19.972145,31.218593,11.246448,15,34,19
3,"[16, 10, 3, 4, 2, 16, 20, 16, 13]",17.576602,28.690955,11.114353,6,32,26
4,"[3, 9, 16, 3, 11, 12, 17, 13, 16]",15.487465,26.361809,10.874344,6,29,23
5,"[3, 2, 15, 9, 4, 16, 13, 17, 19]",16.782730,27.557789,10.775059,12,29,17
6,"[16, 2, 3, 3, 14, 13, 16, 17, 16]",18.066852,28.783920,10.717067,16,32,16
7,"[13, 6, 14, 7, 5, 13, 11, 12, 19]",17.103064,27.771357,10.668293,12,31,19
8,"[11, 7, 11, 8, 6, 15, 11, 13, 16]",17.515320,28.050251,10.534931,14,30,16
9,"[10, 6, 2, 8, 13, 16, 14, 16, 15]",18.961003,29.449749,10.488746,14,32,18


In [383]:
methods = ["is_brightid_verified","is_duniter_verified","is_twitter_verified",
 "is_google_verified","is_facebook_verified","is_poap_verified",
 "is_ens_verified" ,"is_poh_verified","is_idena_verified"]

In [384]:
Num = [i for i in range(9)]

In [387]:
ref = pd.DataFrame()
ref["Methods"] = methods
ref["Weights_index"] = Num

,Methods,Weights_index
0,is_brightid_verified,0
1,is_duniter_verified,1
2,is_twitter_verified,2
3,is_google_verified,3
4,is_facebook_verified,4
5,is_poap_verified,5
6,is_ens_verified,6
7,is_poh_verified,7
8,is_idena_verified,8


In [416]:
import plotly.express as px

combination = []
for i in range(diff.shape[0]):
    string = ",".join([str(item) for item in diff.loc[i, "Combination of weights"]])
    combination.append(string)
    
diff["Weight_combination"] = combination

barchart = px.bar(data_frame= diff,
                 y= "Weight_combination",
                 x= [diff["Mean differences"], diff["Median differences"]],
                 #color="counts",
                 opacity=0.9,
                 orientation="h",
                  color_continuous_scale=px.colors.diverging.Spectral,
                  barmode="group",
                  title="Plot of Mean and median differences of randomized weights for each verification method",
                  labels={"variable": "Class",
                         "x": "Methods", 
                         "true_counts": "True",
                         "false_counts": "False"}
                  #text= verify.counts_True, verify.counts_False
                 #color_discrete_sequence=["darkorange", "lightblue"]
                 ) 

#barchart.update_layout(yaxis = dict(tickmode = 'array', ticktext = [i for i in diff["Combination of weights"]]))

In [417]:
barchart

In [406]:
diff.loc[0, "Combination of weights"]

[16, 2, 3, 3, 14, 13, 16, 17, 16]

In [409]:
combination = []

In [439]:
trust_scores = []

trust_mean_differences = []

trust_median_differences = []

sybil_mean = []

not_sybil_mean = []

sybil_median = []

not_sybil_median = []

def calc_score_no_duniter(dfs):
    
    dff = dfs.copy()
    
    scores = random_with_patterns(4, 4)
    
    trust_scores.append([i for i in scores])
    
    total_score = []
    
    dff.is_brightid_verified = np.where(dff.is_brightid_verified == True, scores[0], 0)
    
    dff.is_twitter_verified = np.where(dff.is_twitter_verified == True, scores[1], 0)
    
    dff.is_google_verified = np.where(dff.is_google_verified == True, scores[2], 0)
    
    dff.is_facebook_verified = np.where(dff.is_facebook_verified == True, scores[3], 0)
    
    dff.is_poap_verified = np.where(dff.is_poap_verified == True, scores[4], 0)
    
    dff.is_ens_verified = np.where(dff.is_ens_verified == True, scores[5], 0)
    
    dff.is_poh_verified = np.where(dff.is_poh_verified == True, scores[6], 0)
    
    dff.is_idena_verified = np.where(dff.is_idena_verified == True, scores[7], 0)

    
    for row in range(dff.shape[0]):
        total_score.append(sum(dff.loc[row, ["is_brightid_verified", "is_twitter_verified", "is_poap_verified",
                             "is_ens_verified", "is_google_verified", 'is_facebook_verified', 'is_poh_verified', 'is_idena_verified']]))
    
    dff["total_score"] = total_score
    
    sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[1]
    not_sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[0]
    
    sybil_mean.append(sybil_mean_)
    not_sybil_mean.append(not_sybil_mean_)
    
    sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[1]
    not_sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[0]
    
    sybil_median.append(sybil_median_)
    not_sybil_median.append(not_sybil_median_)
    
    trust_median_differences.append(abs(sybil_median_ - not_sybil_median_))
    
    trust_mean_differences.append(abs(sybil_mean_ - not_sybil_mean_))

In [440]:
for i in range(15):
    np.random.seed(i)
    calc_score_no_duniter(df)

In [441]:
methods_no_duniter = ["is_brightid_verified","is_twitter_verified",
 "is_google_verified","is_facebook_verified","is_poap_verified",
 "is_ens_verified" ,"is_poh_verified","is_idena_verified"]

Num_no_duniter = [i for i in range(8)]

ref_2 = pd.DataFrame()
ref_2["Methods"] = methods_no_duniter
ref_2["Weights_index"] = Num_no_duniter

In [442]:
ref_2

,Methods,Weights_index
0,is_brightid_verified,0
1,is_twitter_verified,1
2,is_google_verified,2
3,is_facebook_verified,3
4,is_poap_verified,4
5,is_ens_verified,5
6,is_poh_verified,6
7,is_idena_verified,7


In [443]:
## Convert the output to a dataframe for easy overview.
diff2 = pd.DataFrame()
diff2["Combination of weights"] = trust_scores
diff2["Sybil mean"] = sybil_mean
diff2["not_sybil mean"] = not_sybil_mean
diff2["Mean differences"] = trust_mean_differences
diff2["Sybil median"] = sybil_median
diff2["not_sybil median"] = not_sybil_median
diff2["Median differences"] = trust_median_differences

In [448]:
diff2.sort_values(by=["Mean differences", "Median differences"], ascending=False).reset_index(drop=True)

,Combination of weights,Sybil mean,not_sybil mean,Mean differences,Sybil median,not_sybil median,Median differences,Weight_combination
0,"[11, 8, 12, 8, 19, 16, 15, 11]",22.217270,34.987437,12.770167,19.0,39.0,20.0,"11,8,12,8,19,16,15,11"
1,"[17, 10, 6, 9, 13, 19, 12, 14]",20.339833,32.972362,12.632529,15.0,36.0,21.0,"17,10,6,9,13,19,12,14"
2,"[15, 17, 6, 5, 14, 15, 12, 16]",18.899721,31.306533,12.406811,14.0,35.0,21.0,"15,17,6,5,14,15,12,16"
3,"[15, 7, 16, 8, 14, 14, 12, 14]",23.197772,35.540201,12.342429,16.0,39.0,23.0,"15,7,16,8,14,14,12,14"
4,"[3, 9, 17, 4, 21, 14, 18, 14]",21.830084,33.997487,12.167404,20.0,36.5,16.5,"3,9,17,4,21,14,18,14"
5,"[16, 7, 5, 10, 15, 18, 15, 14]",19.738162,31.844221,12.106060,15.0,34.0,19.0,"16,7,5,10,15,18,15,14"
6,"[11, 8, 2, 9, 17, 19, 13, 19]",17.629526,29.195980,11.566453,11.0,31.0,20.0,"11,8,2,9,17,19,13,19"
7,"[5, 5, 15, 10, 15, 16, 17, 15]",21.119777,32.527638,11.407861,15.0,36.0,21.0,"5,5,15,10,15,16,17,15"
8,"[5, 14, 7, 2, 16, 17, 20, 19]",16.791086,28.110553,11.319466,9.0,29.0,20.0,"5,14,7,2,16,17,20,19"
9,"[4, 2, 17, 10, 14, 18, 14, 21]",21.782730,33.055276,11.272547,17.0,35.0,18.0,"4,2,17,10,14,18,14,21"


In [445]:
combination = []

for i in range(diff2.shape[0]):
    string = ",".join([str(item) for item in diff2.loc[i, "Combination of weights"]])
    combination.append(string)
    
diff2["Weight_combination"] = combination

In [446]:
import plotly.express as px
barchart = px.bar(data_frame= diff2,
                 y= "Weight_combination",
                 x= [diff2["Mean differences"], diff2["Median differences"]],
                 #color="counts",
                 opacity=0.9,
                 orientation="h",
                  color_continuous_scale=px.colors.diverging.Spectral,
                  barmode="group",
                  title="Plot of Mean and median differences of randomized weights for each verification method (No Duniter)",
                  labels={"variable": "Class",
                         "x": "Methods", 
                         "true_counts": "True",
                         "false_counts": "False"}
                  #text= verify.counts_True, verify.counts_False
                 #color_discrete_sequence=["darkorange", "lightblue"]
                 ) 

#barchart.update_layout(yaxis = dict(tickmode = 'array', ticktext = [i for i in diff["Combination of weights"]]))

In [447]:
barchart

### Scaling

In [492]:

def calc_score_no_duniter(dfs):
    
    dff = dfs.copy()
    
    scores = random_with_patterns(4, 4)
    
    trust_scores.append([i for i in scores])
    
    total_score = []
    
    dff.is_brightid_verified = np.where(dff.is_brightid_verified == True, scores[0], 0)
    
    dff.is_twitter_verified = np.where(dff.is_twitter_verified == True, scores[1], 0)
    
    dff.is_google_verified = np.where(dff.is_google_verified == True, scores[2], 0)
    
    dff.is_facebook_verified = np.where(dff.is_facebook_verified == True, scores[3], 0)
    
    dff.is_poap_verified = np.where(dff.is_poap_verified == True, scores[4], 0)
    
    dff.is_ens_verified = np.where(dff.is_ens_verified == True, scores[5], 0)
    
    dff.is_poh_verified = np.where(dff.is_poh_verified == True, scores[6], 0)
    
    dff.is_idena_verified = np.where(dff.is_idena_verified == True, scores[7], 0)

    
    for row in range(dff.shape[0]):
        total_score.append(sum(dff.loc[row, ["is_brightid_verified", "is_twitter_verified", "is_poap_verified",
                             "is_ens_verified", "is_google_verified", 'is_facebook_verified', 'is_poh_verified', 'is_idena_verified']]))
    
    dff["total_score"] = total_score
    
    sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[1]
    not_sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[0]
    
    sybil_mean.append(sybil_mean_)
    not_sybil_mean.append(not_sybil_mean_)
    
    sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[1]
    not_sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[0]
    
    sybil_median.append(sybil_median_)
    not_sybil_median.append(not_sybil_median_)
    
    trust_median_differences.append(abs(sybil_median_ - not_sybil_median_))
    
    trust_mean_differences.append(abs(sybil_mean_ - not_sybil_mean_))

In [466]:
df1 = df[df.trust_bonus > 0.5]

In [671]:
df[df.trust_bonus == 0.5]

,handle,Is Sybil? (T or F),trust_bonus,is_brightid_verified,is_duniter_verified,is_twitter_verified,is_poap_verified,is_ens_verified,is_google_verified,is_facebook_verified,is_poh_verified,is_idena_verified
6,gmifrfr,True,0.5,False,False,False,False,False,False,False,False,False
7,ssyahoo,True,0.5,False,False,False,False,False,False,False,False,False
9,phantom5031,True,0.5,False,False,False,False,False,False,False,False,False
12,zpunketh,True,0.5,False,False,False,False,False,False,False,False,False
13,jcarvajalmunera,True,0.5,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
734,yabirgb,False,0.5,False,False,False,False,False,False,False,False,False
740,alpselin,True,0.5,False,False,False,False,False,False,False,False,False
744,cosmiccollusion,True,0.5,False,False,False,False,False,False,False,False,False
749,laobaib,True,0.5,False,False,False,False,False,False,False,False,False


In [468]:
df1.reset_index(drop=True, inplace=True)

In [508]:
#%%timeit

trust_scores = []

trust_mean_differences = []

trust_median_differences = []

sybil_mean = []

not_sybil_mean = []

sybil_median = []

not_sybil_median = []


import time
from datetime import datetime


start = datetime.now()
for i in range(1000):
    np.random.seed(i)
    calc_score_no_duniter(df1)
    
end = datetime.now()

print(end - start)

0:06:09.576951


In [509]:
len(sybil_median)

1000

In [499]:
diff2 = pd.DataFrame()
diff2["Combination of weights"] = trust_scores
diff2["Sybil mean"] = sybil_mean
diff2["not_sybil mean"] = not_sybil_mean
diff2["Mean differences"] = trust_mean_differences
diff2["Sybil median"] = sybil_median
diff2["not_sybil median"] = not_sybil_median
diff2["Median differences"] = trust_median_differences

In [503]:
diff2.sort_values(by="Mean differences", inplace=True, ascending=False)
diff2.reset_index(drop=True, inplace=True)

In [506]:
diff2.to_csv("./first_1k_1dp.csv", index=False)

In [511]:
diff2.head(10)

,Combination of weights,Sybil mean,not_sybil mean,Mean differences,Sybil median,not_sybil median,Median differences
0,"[11.4, 16.9, 3.1, 2.7, 16.1, 20.7, 17.2, 11.9]",30.242925,38.418437,8.175513,33.30,39.9,6.60
1,"[2.9, 16.3, 6.3, 6.7, 15.1, 23.4, 16.7, 12.6]",31.620283,39.465312,7.845029,31.80,41.4,9.60
2,"[14.9, 15.3, 3.0, 3.4, 19.0, 17.2, 15.6, 11.6]",31.414151,39.248750,7.834599,35.10,39.2,4.10
3,"[11.1, 11.5, 2.0, 7.1, 20.2, 20.9, 14.6, 12.6]",31.962264,39.763750,7.801486,32.85,41.3,8.45
4,"[13.2, 16.6, 3.1, 4.7, 14.6, 19.0, 13.9, 14.9]",30.274528,38.044062,7.769534,33.60,37.6,4.00
5,"[13.0, 15.6, 3.6, 2.0, 18.9, 17.6, 11.7, 17.6]",30.757547,38.413438,7.655890,34.20,38.8,4.60
6,"[4.7, 12.4, 3.8, 8.5, 21.4, 21.4, 12.8, 15.0]",32.355189,40.002812,7.647624,33.70,42.8,9.10
7,"[2.5, 18.1, 4.0, 4.8, 21.0, 16.5, 19.0, 14.1]",28.790094,36.398437,7.608343,27.80,40.0,12.20
8,"[4.0, 17.8, 8.0, 4.4, 20.4, 17.8, 13.8, 13.8]",32.924528,40.492500,7.567972,32.80,42.2,9.40
9,"[7.6, 13.1, 2.1, 9.3, 13.1, 23.5, 12.7, 18.6]",29.545283,37.083750,7.538467,32.10,38.7,6.60


In [512]:
import plotly.express as px

combination = []
for i in range(diff2.shape[0]):
    string = ",".join([str(item) for item in diff2.loc[i, "Combination of weights"]])
    combination.append(string)
    
diff2["Weight_combination"] = combination

barchart = px.bar(data_frame= diff2.head(10),
                 y= "Weight_combination",
                 x= [diff2.head(10)["Mean differences"], diff2.head(10)["Median differences"]],
                 #color="counts",
                 opacity=0.9,
                 orientation="h",
                  color_continuous_scale=px.colors.diverging.Spectral,
                  barmode="group",
                  title="Plot of Mean and median differences of randomized weights for each verification method for 1 d.p",
                  labels={"variable": "Class",
                         "x": "Methods", 
                         "true_counts": "True",
                         "false_counts": "False"}
                  #text= verify.counts_True, verify.counts_False
                 #color_discrete_sequence=["darkorange", "lightblue"]
                 ) 

#barchart.update_layout(yaxis = dict(tickmode = 'array', ticktext = [i for i in diff["Combination of weights"]]))

In [513]:
barchart

In [514]:
diff3 = pd.DataFrame()
diff3["Combination of weights"] = trust_scores
diff3["Sybil mean"] = sybil_mean
diff3["not_sybil mean"] = not_sybil_mean
diff3["Mean differences"] = trust_mean_differences
diff3["Sybil median"] = sybil_median
diff3["not_sybil median"] = not_sybil_median
diff3["Median differences"] = trust_median_differences

In [515]:
diff3.sort_values(by="Mean differences", inplace=True, ascending=False)
diff3.reset_index(drop=True, inplace=True)

In [517]:
diff3.head(10)

,Combination of weights,Sybil mean,not_sybil mean,Mean differences,Sybil median,not_sybil median,Median differences
0,"[11.49, 16.86, 3.07, 2.68, 16.09, 20.69, 17.24...",30.229528,38.403500,8.173972,33.33,39.850,6.520
1,"[2.93, 16.32, 6.28, 6.69, 15.06, 23.43, 16.74,...",31.610189,39.462937,7.852749,31.80,41.420,9.620
2,"[14.93, 15.3, 2.99, 3.36, 19.03, 17.16, 15.66,...",31.397972,39.231781,7.833810,35.08,39.180,4.100
3,"[11.07, 11.46, 1.98, 7.11, 20.16, 20.95, 14.62...",31.931698,39.730437,7.798739,32.81,41.305,8.495
4,"[13.22, 16.61, 3.04, 4.75, 14.58, 18.98, 13.9,...",30.235802,38.005719,7.769917,33.56,37.620,4.060
5,"[13.03, 15.64, 3.58, 1.95, 18.89, 17.59, 11.73...",30.731509,38.393344,7.661834,34.20,38.760,4.560
6,"[4.7, 12.39, 3.85, 8.55, 21.36, 21.37, 12.82, ...",32.382830,40.020031,7.637201,33.76,42.730,8.970
7,"[2.42, 18.15, 4.03, 4.84, 20.97, 16.53, 18.95,...",28.807783,36.419531,7.611748,27.82,39.920,12.100
8,"[4.0, 17.78, 8.0, 4.44, 20.44, 17.78, 13.78, 1...",32.947170,40.513000,7.565830,32.88,42.220,9.340
9,"[7.59, 13.08, 2.11, 9.28, 13.08, 23.63, 12.66,...",29.587123,37.140000,7.552877,32.06,38.820,6.760


In [521]:
import plotly.express as px

combination = []
for i in range(diff3.shape[0]):
    string = ",".join([str(item) for item in diff3.loc[i, "Combination of weights"]])
    combination.append(string)
    
diff3["Weight_combination"] = combination

barchart = px.bar(data_frame= diff3.head(10),
                 y= "Weight_combination",
                 x= [diff3.head(10)["Mean differences"], diff3.head(10)["Median differences"]],
                 #color="counts",
                 opacity=0.9,
                 orientation="h",
                  color_continuous_scale=px.colors.diverging.Spectral,
                  barmode="group",
                  title="Plot of Mean and median differences of randomized weights for each verification method for 2 d.p",
                  labels={"variable": "Class",
                         "x": "Methods", 
                         "true_counts": "True",
                         "false_counts": "False"}
                  #text= verify.counts_True, verify.counts_False
                 #color_discrete_sequence=["darkorange", "lightblue"]
                 ) 

#barchart.update_layout(yaxis = dict(tickmode = 'array', ticktext = [i for i in diff["Combination of weights"]]))

In [522]:
barchart

In [523]:
diff3.drop("Weight_combination", axis=1, inplace=True)

In [525]:
diff3.to_csv("./first_1k_for_2dp.csv", index=False)

In [527]:
round(2.3, 0)

2.0

#### Integers 

In [591]:
list(np.random.randint(low = 5, high = 50, size = 5))

[12, 16, 22, 8, 44]

In [593]:
def random_with_patterns_int(n, m):
    
    """
    n: Number of less difficult methods.
    m: Number of more difficult methods.    
    """
    
    ## Generate random values from a lower range for methods with less difficulty
    list1 = list(np.random.randint(low = 5, high = 50, size = n))
    
    ## Generate random values from a higher range for more difficult verification methods
    list2 = list(np.random.randint(low = 30, high = 60, size=m))
    
    ## Join both list
    for i in list2:
        list1.append(i)
        
    ## Convert them, such that they sum up to 100
    list_ = [int(round(i/sum(list1)) * 100) for i in list1]
    
    ## Handling the errors due to approximation during rounding up.
    if sum(list_) > 100:
        
        ## A random index where the correction will be made.
        my_index = np.random.randint(len(list_))
        
        ## Subtract the excessive value from the selected index
        list_[my_index] = list_[my_index] - (sum(list_) - 100)
        
        ## Returns the list
        return list_
    
    ## Handling values that sums up to less than 100 after rounding up
    elif sum(list_) < 100:
        
        ## Select random index.
        my_index = np.random.randint(len(list_))
        
        ## Add the remaining to the selected index.
        list_[my_index] = list_[my_index] + (100 - sum(list_))
        
        ## Return the list
        return list_
    
    ## The value equals 100
    else:
        return list_

In [600]:
def random_int_generator(n, m):
    
    """
    n: Number of less difficult methods.
    m: Number of more difficult methods.    
    """
    
    ## Generate random values from a lower range for methods with less difficulty
    list1 = list(np.random.randint(low = 5, high = 50, size = n))
    
    ## Generate random values from a higher range for more difficult verification methods
    list2 = list(np.random.randint(low = 30, high = 60, size=m))
    
    ## Join both list
    for i in list2:
        list1.append(i)
        
    ## Convert them, such that they sum up to 100
    list_ = [int(round(i/sum(list1) * 100)) for i in list1]
    
    ## Handling the errors due to approximation during rounding up.
    if sum(list_) > 100:
        
        ## A random index where the correction will be made.
        my_index = np.random.randint(len(list_))
        
        ## Subtract the excessive value from the selected index
        list_[my_index] = list_[my_index] - (sum(list_) - 100)
        
        ## Returns the list
        return list_
    
    ## Handling values that sums up to less than 100 after rounding up
    elif sum(list_) < 100:
        
        ## Select random index.
        my_index = np.random.randint(len(list_))
        
        ## Add the remaining to the selected index.
        list_[my_index] = list_[my_index] + (100 - sum(list_))
        
        ## Return the list
        return list_
    
    ## The value equals 100
    else:
        return list_

In [616]:
random_int_generator(5, 4)

[1, 14, 9, 6, 7, 18, 14, 18, 13]

In [617]:

def calc_score_no_duniter_int(dfs):
    
    dff = dfs.copy()
    
    scores = random_int_generator(4, 4)
    
    trust_scores.append([i for i in scores])
    
    total_score = []
    
    dff.is_brightid_verified = np.where(dff.is_brightid_verified == True, scores[0], 0)
    
    dff.is_twitter_verified = np.where(dff.is_twitter_verified == True, scores[1], 0)
    
    dff.is_google_verified = np.where(dff.is_google_verified == True, scores[2], 0)
    
    dff.is_facebook_verified = np.where(dff.is_facebook_verified == True, scores[3], 0)
    
    dff.is_poap_verified = np.where(dff.is_poap_verified == True, scores[4], 0)
    
    dff.is_ens_verified = np.where(dff.is_ens_verified == True, scores[5], 0)
    
    dff.is_poh_verified = np.where(dff.is_poh_verified == True, scores[6], 0)
    
    dff.is_idena_verified = np.where(dff.is_idena_verified == True, scores[7], 0)

    
    for row in range(dff.shape[0]):
        total_score.append(sum(dff.loc[row, ["is_brightid_verified", "is_twitter_verified", "is_poap_verified",
                             "is_ens_verified", "is_google_verified", 'is_facebook_verified', 'is_poh_verified', 'is_idena_verified']]))
    
    dff["total_score"] = total_score
    
    sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[1]
    not_sybil_mean_ = dff.groupby("Is Sybil? (T or F)")["total_score"].mean()[0]
    
    sybil_mean.append(sybil_mean_)
    not_sybil_mean.append(not_sybil_mean_)
    
    sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[1]
    not_sybil_median_ = dff.groupby("Is Sybil? (T or F)")["total_score"].median()[0]
    
    sybil_median.append(sybil_median_)
    not_sybil_median.append(not_sybil_median_)
    
    trust_median_differences.append(abs(sybil_median_ - not_sybil_median_))
    
    trust_mean_differences.append(abs(sybil_mean_ - not_sybil_mean_))

In [685]:
trust_scores = []

trust_mean_differences = []

trust_median_differences = []

sybil_mean = []

not_sybil_mean = []

sybil_median = []

not_sybil_median = []


import time
from datetime import datetime


start = datetime.now()
for i in range(1000):
    np.random.seed(i)
    calc_score_no_duniter_int(df1)
    
end = datetime.now()

print(end - start)

0:18:22.925293


In [694]:
comb = pd.DataFrame()
comb["Combination"] = [1, 10, 100, 1000, 10000, 100000, 1000000]
comb["Time"] = ["0:00:00.098666", "0:00:10.601749", "0:01:47.152032", "0:18:22.925293",
                "1:40:33.186432", "6:10:28.876507", "11:23:04.548901"]

In [695]:
comb

,Combination,Time
0,1,0:00:00.098666
1,10,0:00:10.601749
2,100,0:01:47.152032
3,1000,0:18:22.925293
4,10000,1:40:33.186432
5,100000,6:10:28.876507
6,1000000,11:23:04.548901


In [698]:
x = pd.DataFrame()
x["X"] =comb.Combination.value_counts().index
x["Y"] = comb.Combination.value_counts().values
x

,X,Y
0,10,1
1,1000000,1
2,1000,1
3,100000,1
4,100,1
5,1,1
6,10000,1


In [700]:
True + True 

2

In [622]:
diff4 = pd.DataFrame()
diff4["Combination of weights"] = trust_scores
diff4["Sybil mean"] = sybil_mean
diff4["not_sybil mean"] = not_sybil_mean
diff4["Mean differences"] = trust_mean_differences
diff4["Sybil median"] = sybil_median
diff4["not_sybil median"] = not_sybil_median
diff4["Median differences"] = trust_median_differences

In [702]:
pd.options.display.max_rows = None

In [703]:
diff4.

,Combination of weights,Sybil mean,not_sybil mean,Mean differences,Sybil median,not_sybil median,Median differences
0,"[11, 17, 3, 3, 16, 21, 17, 12]",30.221698,38.431250,8.209552,33.0,40.0,7.0
1,"[13, 17, 2, 5, 15, 19, 14, 15]",29.655660,37.556250,7.900590,34.0,37.0,3.0
2,"[13, 16, 4, 2, 19, 18, 12, 16]",31.410377,39.206250,7.795873,35.0,40.0,5.0
3,"[15, 15, 3, 3, 19, 17, 16, 12]",31.136792,38.896875,7.760083,35.0,39.0,4.0
4,"[3, 16, 6, 7, 15, 23, 17, 13]",31.235849,38.984375,7.748526,31.5,41.0,9.5
5,"[11, 11, 2, 7, 20, 21, 15, 13]",31.721698,39.440625,7.718927,32.5,41.0,8.5
6,"[2, 18, 4, 5, 21, 17, 19, 14]",28.867925,36.515625,7.647700,28.0,40.0,12.0
7,"[5, 7, 4, 2, 24, 25, 16, 17]",31.759434,39.359375,7.599941,31.5,40.5,9.0
8,"[4, 18, 8, 4, 20, 18, 14, 14]",32.698113,40.287500,7.589387,32.0,42.0,10.0
9,"[4, 16, 5, 7, 15, 21, 19, 13]",29.886792,37.443750,7.556958,31.0,40.0,9.0


In [624]:
diff4.sort_values(by="Mean differences", inplace=True, ascending=False)
diff4.reset_index(drop=True, inplace=True)

In [632]:
diff4.head(10)

,Combination of weights,Sybil mean,not_sybil mean,Mean differences,Sybil median,not_sybil median,Median differences
0,"[11, 17, 3, 3, 16, 21, 17, 12]",30.221698,38.431250,8.209552,33.0,40.0,7.0
1,"[13, 17, 2, 5, 15, 19, 14, 15]",29.655660,37.556250,7.900590,34.0,37.0,3.0
2,"[13, 16, 4, 2, 19, 18, 12, 16]",31.410377,39.206250,7.795873,35.0,40.0,5.0
3,"[15, 15, 3, 3, 19, 17, 16, 12]",31.136792,38.896875,7.760083,35.0,39.0,4.0
4,"[3, 16, 6, 7, 15, 23, 17, 13]",31.235849,38.984375,7.748526,31.5,41.0,9.5
5,"[11, 11, 2, 7, 20, 21, 15, 13]",31.721698,39.440625,7.718927,32.5,41.0,8.5
6,"[2, 18, 4, 5, 21, 17, 19, 14]",28.867925,36.515625,7.647700,28.0,40.0,12.0
7,"[5, 7, 4, 2, 24, 25, 16, 17]",31.759434,39.359375,7.599941,31.5,40.5,9.0
8,"[4, 18, 8, 4, 20, 18, 14, 14]",32.698113,40.287500,7.589387,32.0,42.0,10.0
9,"[4, 16, 5, 7, 15, 21, 19, 13]",29.886792,37.443750,7.556958,31.0,40.0,9.0


In [628]:
import plotly.express as px

combination = []
for i in range(diff4.shape[0]):
    string = " , ".join([str(item) for item in diff4.loc[i, "Combination of weights"]])
    combination.append(string)
    
diff4["Weight_combination"] = combination

barchart = px.bar(data_frame= diff4.head(10),
                 y= "Weight_combination",
                 x= [diff4.head(10)["Mean differences"], diff4.head(10)["Median differences"]],
                 #color="counts",
                 opacity=0.9,
                 orientation="h",
                  color_continuous_scale=px.colors.diverging.Spectral,
                  barmode="group",
                  title="Plot of Mean and median differences of randomized weights for each verification method for integers",
                  labels={"variable": "Class",
                         "x": "Methods", 
                         "true_counts": "True",
                         "false_counts": "False"}
                  #text= verify.counts_True, verify.counts_False
                 #color_discrete_sequence=["darkorange", "lightblue"]
                 ) 

#barchart.update_layout(yaxis = dict(tickmode = 'array', ticktext = [i for i in diff["Combination of weights"]]))

In [629]:
barchart

In [630]:
diff4.drop("Weight_combination", axis=1, inplace=True)

In [631]:
diff4.to_csv("./weights_for_first_1k_int_no_dp.csv", index=False)

In [665]:
matrix = []
for i in range(10):
    np.random.seed(i)
    matrix.append(np.random.randint(low=5, high=10, size = 6))

In [662]:
matrix1 =np.asarray(matrix)

In [666]:
for i in matrix:
    print(i * np.array(i).T)

[81 25 64 64 64 36]
[64 81 25 36 64 25]
[25 25 64 49 64 25]
[49 25 36 64 25 25]
[49 36 25 25 49 36]
[64 25 36 25 81 64]
[49 36 64 81 49 49]
[81 36 64 64 81 36]
[64 81 36 36 49 25]
[81 25 64 36 81 64]


In [667]:
matrix

[array([9, 5, 8, 8, 8, 6]),
 array([8, 9, 5, 6, 8, 5]),
 array([5, 5, 8, 7, 8, 5]),
 array([7, 5, 6, 8, 5, 5]),
 array([7, 6, 5, 5, 7, 6]),
 array([8, 5, 6, 5, 9, 8]),
 array([7, 6, 8, 9, 7, 7]),
 array([9, 6, 8, 8, 9, 6]),
 array([8, 9, 6, 6, 7, 5]),
 array([9, 5, 8, 6, 9, 8])]

In [672]:
np.random.choice([True, False], 8)

array([ True,  True, False,  True, False, False, False, False])

In [673]:
df2 = df1.copy()

In [674]:
x = 0
while x < 10:
    df2 = pd.concat([df2, df1])
    x += 1

In [675]:
df2.shape

(5852, 12)

In [ ]:
d